In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 5-3. ポリモーフィズム

### 5-3-1. Julia のポリモーフィズム

#### コード5-17.	アドホック多相の例（「鳴く」のポリモーフィズムを Julia のコードで再現）

In [2]:
struct 犬 end

In [3]:
struct ネコ end

In [4]:
struct ミンミンゼミ end

In [5]:
鳴く(::犬) = "ワン"

鳴く (generic function with 1 method)

In [6]:
鳴く(::ネコ) = "ニャー"

鳴く (generic function with 2 methods)

In [7]:
鳴く(::ミンミンゼミ) = "ミーンミーン"

鳴く (generic function with 3 methods)

In [8]:
鳴く(犬())

"ワン"

In [9]:
鳴く(ネコ())

"ニャー"

In [10]:
鳴く(ミンミンゼミ())

"ミーンミーン"

#### コード5-18.	パラメータ多相の例 (1): 定義例（`predtypes()`）

In [11]:
function predtypes(x::Int, y::Int)
    "Both $x and $y are `Int`."
end

predtypes (generic function with 1 method)

In [12]:
function predtypes(x::T, y::T) where T
    "Both $x and $y are the same types ($T)."
end

predtypes (generic function with 2 methods)

In [13]:
function predtypes(x::N, y::N) where {N <: Number}
    "Both $x and $y are the same Number types ($N)."
end

predtypes (generic function with 3 methods)

In [14]:
function predtypes(x::T1, y::T2) where {T1, T2}
    "$x(::$T1) and $y(::$T2) are diffrent types."
end

predtypes (generic function with 4 methods)

#### コード5-19.	パラメータ多相の例 (2): 実行例（`predtypes()` の動作確認）

In [15]:
Base.typesof(1, 2)

Tuple{Int64, Int64}

In [16]:
predtypes(1, 2)

"Both 1 and 2 are `Int`."

In [17]:
Base.typesof("abc", "漢字")

Tuple{String, String}

In [18]:
predtypes("abc", "漢字")

"Both abc and 漢字 are the same types (String)."

In [19]:
Base.typesof(:ok, :NG)

Tuple{Symbol, Symbol}

In [20]:
predtypes(:ok, :NG)

"Both ok and NG are the same types (Symbol)."

In [21]:
Base.typesof(1.0, 2.2)

Tuple{Float64, Float64}

In [22]:
predtypes(1.0, 2.2)

"Both 1.0 and 2.2 are the same Number types (Float64)."

In [23]:
Base.typesof(1//2, 3//4)

Tuple{Rational{Int64}, Rational{Int64}}

In [24]:
predtypes(1//2, 3//4)

"Both 1//2 and 3//4 are the same Number types (Rational{Int64})."

In [25]:
Base.typesof(1.0, π)

Tuple{Float64, Irrational{:π}}

In [26]:
predtypes(1.0, π)

"1.0(::Float64) and π(::Irrational{:π}) are diffrent types."

### 5-3-2. 実例

#### コード5-20.	GeometricSequence.jl (1): `Base.show()` の多重定義まで

In [27]:
struct GeometricSequence{T<:Number} <: AbstractVector{T}
    a::T
    r::T
    n::Int
end

function GeometricSequence(a::T1, r::T2, n::Integer) where {T1<:Number, T2<:Number}
    GeometricSequence(promote(a, r)..., Int(n))
end

Base.length(seq::GeometricSequence) = seq.n
Base.size(seq::GeometricSequence) = (seq.n,)
Base.getindex(seq::GeometricSequence, index::Integer) = seq.a * seq.r ^ (index - 1)

function Base.show(io::IO, seq::GeometricSequence)
    print(io, "GeometricSequence(", seq.a, ", ", seq.r, ", ", seq.n, ")")
end
Base.show(io::IO, ::MIME"text/plain", seq::GeometricSequence) = show(io, seq)

#### コード5-21.	`GeometricSequence` の動作確認 (1): 基本的な動作

In [28]:
seq0 = GeometricSequence(1, 3, 5)

GeometricSequence(1, 3, 5)

In [29]:
length(seq0)

5

In [30]:
size(seq0)

(5,)

In [31]:
collect(seq0)

5-element Vector{Int64}:
  1
  3
  9
 27
 81

In [32]:
sum(seq0)

121

#### コード5-22. GeometricSequence.jl (2): Base.sum() の多重定義（ポリモーフィズム）

In [33]:
function Base.sum(seq::GeometricSequence)
    seq.a * (1 - seq.r^seq.n) / (1 - seq.r)
end

function Base.sum(seq::GeometricSequence{<:Rational})
    c = numerator(seq.a)
    b = denominator(seq.a)
    q = numerator(seq.r)
    p = denominator(seq.r)
    n = seq.n
    c * (p^n - q^n) // (b * p^(n-1) * (p - q))
end

function Base.sum(seq::GeometricSequence{<:Integer})
    seq.a * ((seq.r^seq.n - 1) ÷ (seq.r - 1))
end

#### コード5-23.	`GeometricSequence` の動作確認 (2): `sum()` の動作

In [34]:
sum_naive(seq) = reduce(+, seq)

sum_naive (generic function with 1 method)

In [35]:
sum_with_formula(seq) = seq.a * (1 - seq.r^seq.n) / (1 - seq.r)

sum_with_formula (generic function with 1 method)

In [36]:
seq1 = GeometricSequence(1, 0.5, 20)

GeometricSequence(1.0, 0.5, 20)

In [37]:
(sum_naive(seq1), sum(seq1))

(1.9999980926513672, 1.9999980926513672)

In [38]:
using BenchmarkTools

In [39]:
@btime sum_naive($seq1)

  109.150 ns (0 allocations: 0 bytes)


1.9999980926513672

In [40]:
@btime sum($seq1)

  8.636 ns (0 allocations: 0 bytes)


1.9999980926513672

In [41]:
seq2 = GeometricSequence(1, 1//2, 20)

GeometricSequence(1//1, 1//2, 20)

In [42]:
(sum_naive(seq2), sum_with_formula(seq2), sum(seq2))

(1048575//524288, 1048575//524288, 1048575//524288)

In [43]:
@btime sum_naive($seq2)

  2.144 μs (0 allocations: 0 bytes)


1048575//524288

In [44]:
@btime sum_with_formula($seq2)

  176.253 ns (0 allocations: 0 bytes)


1048575//524288

In [45]:
@btime sum($seq2)

  64.896 ns (0 allocations: 0 bytes)


1048575//524288

In [46]:
seq3 = GeometricSequence(1, 3, 20)

GeometricSequence(1, 3, 20)

In [47]:
(sum_naive(seq3), sum_with_formula(seq3), sum(seq3), sum_naive(seq3) == sum_with_formula(seq3) == sum(seq3))

(1743392200, 1.7433922e9, 1743392200, true)

In [48]:
typeof.((sum_naive(seq3), sum_with_formula(seq3), sum(seq3)))

(Int64, Float64, Int64)

In [49]:
@btime sum_naive($seq3)

  79.760 ns (0 allocations: 0 bytes)


1743392200

In [50]:
@btime sum_with_formula($seq3)

  5.613 ns (0 allocations: 0 bytes)


1.7433922e9

In [51]:
@btime sum($seq3)

  7.652 ns (0 allocations: 0 bytes)


1743392200